# 📌 Introdução

Neste arquivo vamos:
  
1- Ler e interpretar a página em html;

2- Encontrar os dados que queremos raspar;

3- Tratar e exportar os dados para usarmos posteriormente.

# 📥 Objetivo

Este documento realiza o **web scraping** dos dados de imóveis utilizados na análise de preços.

# 🔗 Fonte dos Dados

Os dados foram coletados do site https://www.nutristrategy.com/caloriesburned.htm por meio da biblioteca `rvest`.

## 💻 Código

Instalando e Carregando os Pacotes necessários


In [6]:
if (!require(rvest)) install.packages("rvest")
if (!require(stringr)) install.packages("stringr")
if (!require(tidyverse)) install.packages("tidyverse")
if (!require(broom)) install.packages("broom")
if (!require(ggthemes)) install.packages("ggthemes")
if (!require(patchwork)) install.packages("patchwork")


Carregando a página que vamos raspar os dados



In [7]:
url <- "https://www.nutristrategy.com/caloriesburned.htm"

html <- read_html(url)

tabela <- html |> 
  html_elements("table") |> 
  html_table()

dados <- tabela[[4]]
head(dados)


X1,X2,X3,X4,X5
<chr>,<chr>,<chr>,<chr>,<chr>
"Activity, Exercise or Sport (1 hour)",130 lb,155 lb,180 lb,205 lb
"Cycling, mountain bike, bmx",502,598,695,791
"Cycling, <10 mph, leisure bicycling",236,281,327,372
"Cycling, >20 mph, racing",944,1126,1308,1489
"Cycling, 10-11.9 mph, light",354,422,490,558
"Cycling, 12-13.9 mph, moderate",472,563,654,745


Raspando os dados que vamos utilizar e montando o dataframe



In [8]:
conversor <- function(l) {   # Converte libras em quilos
  kg <- round(l/2.205,0)
  return(kg)
}

colnames(dados) <- c("atividade", conversor(c(130,155,180,205)))

dados <- dados[-1,]

dados <- dados |> 
  pivot_longer(
    cols = -atividade,
               names_to = "peso",
               values_to = "caloria") |> 
  mutate(
    peso = as.numeric(peso),
    atividade = str_squish(atividade),
    caloria = as.numeric(caloria)
  )

str(dados)


tibble [996 × 3] (S3: tbl_df/tbl/data.frame)
 $ atividade: chr [1:996] "Cycling, mountain bike, bmx" "Cycling, mountain bike, bmx" "Cycling, mountain bike, bmx" "Cycling, mountain bike, bmx" ...
 $ peso     : num [1:996] 59 70 82 93 59 70 82 93 59 70 ...
 $ caloria  : num [1:996] 502 598 695 791 236 ...


Salvando os dados



In [10]:
write.csv(dados, "../datas/dados_raspados.csv", row.names = F)



Podemos seguir com a análise e modelagem

